In [ ]:
import ast
import re
import json
from openai import BadRequestError 
from kiwipiepy.utils import Stopwords # 한국어 자연어 처리 패키지
from kiwipiepy import Kiwi
import faiss
import numpy as np
import pandas as pd
from openai import OpenAI

from langchain.prompts import FewShotPromptTemplate, PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import OpenAI as OpenAI2

from hard_prompt import classify_prompt, purchase_prompt, get_chat_prompt, RAG_prompt, matching_rag_prompt

client = OpenAI(api_key = api_key)

In [2]:
def get_embedding(text, model="text-embedding-3-large"):
    text = str(text).replace("\n", " ")
    return client.embeddings.create(input=[text], model=model).data[0].embedding

# 텍스트 벡터값으로 변환 (array 형식으로 맞추기)
def to_vector(x):
    _vector_ = get_embedding(x)
    _vector = np.array([_vector_]).astype(np.float32)
    return _vector

In [3]:
df = pd.DataFrame({'Agent':['반찬의달인', '대박쭈꾸미', '똘이상회', '대유수산'],
                             '메시지':['반찬의달인님이 중부시장테스트가맹점에서 진미채 500g을 1개 당 15000원에 판매하고 있습니다.',
                                    '대박쭈꾸미님이 중부시장테스트가맹점13에서 젤리 100g을 1개 당 8000원에 판매하고 있습니다.',
                                    '똘이상회님이 똘이상회에서 한입에 쏙 젤리(1kg)을 1개 당 10000원에 판매하고 있습니다.',
                                    '대유수산님이 대유수산에서 햇곱창돌김(100장) 450g을 1개 당 27900원에 판매하고 있습니다.'],
                             'Embeddings':[get_embedding('반찬의달인님이 중부시장테스트가맹점에서 진미채 500g을 1개 당 15000원에 판매하고 있습니다.'),
                                           get_embedding('대박쭈꾸미님이 중부시장테스트가맹점13에서 젤리 100g을 1개 당 8000원에 판매하고 있습니다.'),
                                           get_embedding('똘이상회님이 똘이상회에서 한입에 쏙 젤리(1kg)을 1개 당 10000원에 판매하고 있습니다.'),
                                           get_embedding('대유수산님이 대유수산에서 햇곱창돌김(100장) 450g을 1개 당 27900원에 판매하고 있습니다.')]})

In [4]:
vectors_list = df['Embeddings'].tolist()

In [5]:
## 4/2 수정
def purchase_keyword(msg):
    example_prompt = PromptTemplate(
        input_variables=["question", "answer"],
        template="question: {question}\nanswer: {answer}"
    )
    prompt = FewShotPromptTemplate(
        examples=purchase_prompt,
        example_prompt=example_prompt,
        suffix="question : {question}",
        input_variables=["question"],
    )
    llm = OpenAI2(api_key=api_key)
    parser = StrOutputParser()
    chain = prompt | llm | parser
    chain_answer = chain.invoke({"question": msg})
    try:
        answer_list = ast.literal_eval(str(chain_answer[chain_answer.find('form: ')+6:]))

        if len(answer_list) == 1:
            return 'no_product'
        elif len(answer_list) == 2:
            if answer_list[0] == 'no_product' or answer_list[0] == '':
                return 'no_product'
            else:
                return answer_list
        elif answer_list[1] == '' and answer_list[2] == '':
            return 'no_product'
        else:
            return 'no_product'
    except SyntaxError:
        return 'no_product'

In [6]:
## 4/2 수정
def classify_process(text, state):
    prompt = f"""Once, Rephrase the user's sentence({text}) in a standard language.
                Remember that there is a nickname of someone.

                If the sentence seems to pay or buy something with certainty, only print 'Buy&Sell'.
                If the sentence seems to be connected to(by chatting), or matched with someone, only print 'Match'.
                If you cannot judge, or the sentence is a kind of questions or seemes to wnat info, only print out 'Chat'.

                When you judge, please check the sentence the context with the last chat: {state[-1]}
                Also, think about why you judge like that."""
    response = client.chat.completions.create(
                    model = "gpt-4-1106-preview",
                    messages=[{"role": "system", "content": prompt}],
                    temperature = 0
                )
    response = response.choices[0].message.content
    return response

In [7]:
def ends_with_jong(kstr):
    k = kstr[-1]
    if "가" <= k <= "힣":
        if (ord(k)-ord("가")) % 28 > 0:
            return "과"
        else:
            return "와"
    else:
        return "와"

In [8]:
def message_process(msg, state, matched_agent, matched_msg):
    state.append({"role": "user", "content": msg})

    if not msg:
        empty_msg = '대화를 입력해주세요.'
        state.append({"role": "assistant", "content": empty_msg})
        result_ME = { "ai_flag": "M", "type": 'chat', "mem_nick_name": '', "product_name": '', "product_price": 0, "talk_msg": msg}
        result_AI = { "ai_flag": "A", "type": 'chat', "mem_nick_name": '', "product_name": '', "product_price": 0, "talk_msg": empty_msg}
        return "end", result_ME, result_AI, state

    else:
        
        for i in range(len(matched_agent)):
            if matched_agent[i] in msg.replace(" ", ""):
                ## 4/2 수정
                classified_process = classify_process(msg, state)
                print('classified_process: ' , classified_process)
                # 채팅창 연결 process로 이동, 최종 연결된 Agent 및 메시지 return
                if classified_process == 'Match':
                    consonant = ends_with_jong(matched_agent[i])
                    connection_msg = f"네 {matched_agent[i]+consonant} 연결해드리겠습니다."
                    state.append({"role": "assistant", "content": connection_msg})
                    message = matched_msg[i]
                    connection_keyword = purchase_keyword(message)
                    if connection_keyword == 'no_product':
                        result_ME = { "ai_flag": "M", "type": 'connect', "mem_nick_name": matched_agent[i], "product_name": '', "product_price": 0, "talk_msg": msg}
                        result_AI = { "ai_flag": "A", "type": 'connect', "mem_nick_name": matched_agent[i], "product_name": '', "product_price": 0, "talk_msg": connection_msg}
                    else:
                        result_ME = { "ai_flag": "M", "type": 'connect', "mem_nick_name": matched_agent[i], "product_name": connection_keyword[0], "product_price": connection_keyword[1], "talk_msg": msg}
                        result_AI = { "ai_flag": "A", "type": 'connect', "mem_nick_name": matched_agent[i], "product_name": connection_keyword[0], "product_price": connection_keyword[1], "talk_msg": connection_msg}
                    return "end", result_ME, result_AI, state

                # 결제 process로 이동, 최종 연결된 Agent 및 메시지 return
                elif classified_process == 'Buy&Sell':
                    ## 3/22 수정(윤이지): 여기 db에서 꺼내오는걸로 수정 > Agent명 중복 방지 위해 matched_msg로 수정
                    message = matched_msg[i]
                    product_keyword = purchase_keyword(message)
                    if product_keyword == 'no_product':
                        answer_msg = '결제할 상품이 없습니다.'
                        state.append({"role": "assistant", "content": answer_msg})
                        result_ME = { "ai_flag": "M", "type": 'chat', "mem_nick_name": '', "product_name": '', "product_price": 0, "talk_msg": msg}
                        result_AI = { "ai_flag": "A", "type": 'chat', "mem_nick_name": '', "product_name": '', "product_price": 0, "talk_msg": answer_msg}
                        return "end", result_ME, result_AI, state
                    else:
                        pay_nickname = matched_agent[i]
                        pay_product_name =  product_keyword[0]
                        pay_product_price = product_keyword[1]
                        # pay_product_quantity = product_keyword[2]
                        pay_msg = "결제를 진행합니다"
                        state.append({"role": "assistant", "content": pay_msg})
                        result_ME = { "ai_flag": "M", "type": 'sale', "mem_nick_name": pay_nickname, "product_name": pay_product_name, "product_price": pay_product_price, "talk_msg": msg}
                        result_AI = { "ai_flag": "A", "type": 'sale', "mem_nick_name": pay_nickname, "product_name": pay_product_name, "product_price": pay_product_price, "talk_msg": pay_msg}
                    return "end", result_ME, result_AI, state
                
                ## 3/28 수정(2)
                elif classified_process == 'Chat':
                    state.append({"role": "system", "content": f"{msg}에 한국어로 답변하고, 없는 말을 지어내지 마라."})
                    response = client.chat.completions.create(
                    model="gpt-4-1106-preview",
                    messages=state,
                    stream=False)
                    response = response.choices[0].message.content
                    result_ME = { "ai_flag": "M", "type": 'chat', "mem_nick_name": '', "product_name": '', "product_price": 0, "talk_msg": msg}
                    result_AI = { "ai_flag": "A", "type": 'chat', "mem_nick_name": '', "product_name": '', "product_price": 0, "talk_msg": response}                   
                    return "end", result_ME, result_AI, state
    
    # 통상적인 경우 "next" 문자열을 통해 다음 process 실행
        return "next", '-', '-', state

In [9]:
kiwi = Kiwi(typos="basic", model_type='sbg')
stopwords = Stopwords()
stopwords.remove(('사람', 'NNG'))

In [10]:
def tokenize_N(text):
    split_s = kiwi.tokenize(text, stopwords=stopwords, normalize_coda=True)
    N_list = [i.form for i in split_s if i.tag == "NNG" or i.tag == "NNP"]
    split_list = [i.form for i in split_s]
    split_f = ','.join(split_list).replace(",", " ")
    return split_f, N_list

In [11]:
def faiss_index(vectors_list):
    vectors = np.array(vectors_list, dtype=np.float32)
    if vectors.size == 0:
        vectors = np.zeros((1, 3072), dtype=np.float32)

    index = faiss.IndexFlatL2(vectors.shape[1]) 
    index.add(vectors)
    return index

index = faiss_index(vectors_list)

def keyword_search(keyword):
    final_list = []
    k_search = df[df['메시지'].str.contains(keyword)].index
    if len(k_search) > 1:
        n = 0
        while len(final_list) < 2:
            final_list += [(df.iloc[k_search[n], 0], df.iloc[k_search[n], 1])]
            n += 1
    elif len(k_search) == 1:
        final_list += [(df.iloc[k_search[0], 0], df.iloc[k_search[0], 1])]
    else:
        pass
    return final_list

# 입력 문장과 유사한 문장들 검색
def extract_sentences(indices, exclude_nickname=None):
    extracted_info = []
    for idx in indices.flatten():
        if idx < len(df):
            agent_name = df.iloc[idx]['Agent']
            message = df.iloc[idx]['메시지']
            # 사용자의 nickname을 포함하는 항목을 제외
            if exclude_nickname is None or agent_name != exclude_nickname:
                extracted_info.append((agent_name, message))
    return extracted_info

# 기존 list 내에 있는 문장을 제외하고 추가로 검색
# k1은 처음 검색해올 문장 개수, k2는 최종적으로 출력할 문장 개수(중복된 문장이 몇개인지에 따라 달라짐)
def search_vector(sentence, k1, k2, dp_list):
    list_ = []
    _, indices = index.search(to_vector(sentence), k=k1)
    extracted_info = extract_sentences(indices)
    for info in extracted_info[:k2]:
        if info not in dp_list:
            list_.append(info)
    return list_

In [12]:
def find_closest_match(msg_type, msg_object, user_input, nickname, number):
    final_list = []
    try:
        tokenized_input, n_list = tokenize_N(user_input)
        n_list = n_list[:number[0]]
        # 4/2 수정 -eunjeong 
        if msg_type == "Buy&Sell" or msg_type == 'Payment':
            for n_word in n_list:
                final_list += keyword_search(n_word)
            final_list = list(set(final_list))
            # final_list.append('-----------------------------------------keyword-----------------------------------------')
            final_list += search_vector(msg_object, number[1]*2, number[1], final_list)
            # final_list.append('------------------------------------object_sentence--------------------------------------')
            final_list += search_vector(tokenized_input, number[2]*2, number[2], final_list)
            # final_list.append('-----------------------------------tokenized_sentence-------------------------------------')
            _, indices = index.search(to_vector(user_input), k=number[3])
            extracted_info = extract_sentences(indices)
            n = 0
            while len(final_list) < number[3]:
                if extracted_info[n] not in final_list:
                    final_list.append(extracted_info[n])
                n += 1
        # 4/2 수정 -eunjeong 
        elif msg_type == "Match" or msg_type == 'Connect':
            final_list += search_vector(msg_object, number[1]*2, number[1], final_list)
            # final_list.append('---------------object------------------')
            final_list += search_vector(tokenized_input, number[2]*2, number[2], final_list)
            # final_list.append('---------------tokenized----------------')
            _, indices = index.search(to_vector(user_input), k=number[3])
            extracted_info = extract_sentences(indices)
            n = 0
            while len(final_list) < number[3]:
                if extracted_info[n] not in final_list:
                    final_list.append(extracted_info[n])
                n += 1
        else:
            for n_word in n_list:
                final_list += keyword_search(n_word)
            final_list = list(set(final_list))
            # final_list.append('---------------keyword------------------')
            final_list += search_vector(tokenized_input, number[2]*2, number[2], final_list)
            # final_list.append('---------------tokenized----------------')
            _, indices = index.search(to_vector(user_input), k=number[3])
            extracted_info = extract_sentences(indices)
            n = 0
            while len(final_list) < number[3]:
                if extracted_info[n] not in final_list:
                    final_list.append(extracted_info[n])
                n += 1
    except BadRequestError:
        _, indices = index.search(to_vector(user_input), k=number[3])
        final_list = extract_sentences(indices)
        
    except IndexError:
        final_list = [(x, y) for x, y in zip(df['Agent'], df['메시지'])]
    return final_list

In [24]:
def classify(msg, matched_agent, state):
    # 활동명만 입력했을 경우 Match나 Buy&Sell로 연결되는 것을 방지하기 위함
    if msg.replace(" ", "") in matched_agent:
        return ["Chat"]
    else:
        prompt = classify_prompt(msg, state)

        response = client.chat.completions.create(
                        model = "gpt-4-1106-preview",
                        messages=[{"role": "system", "content": prompt}],
                        temperature = 0
                    )
        response = response.choices[0].message.content
        try:
            classification = ast.literal_eval(response)
            print(state[-2:])
            print(classification)
            if classification[0] == "Buy&Sell":
                return ["Buy&Sell", classification[1]]
            elif classification[0] == "Match":
                return ["Match", classification[1]]
            # 4/2 수정 -eunjeong 
            elif classification[0] == "Connect":
                return ["Connect", classification[1]]
            elif classification[0] == "Payment":
                return ["Payment", classification[1]]
            else:
                return ["Chat"]
        except:
            return ["Chat"]

In [14]:
def make_profile(nickname, method, msg, matched_agent, number, state):
    if method == 0:
        profile = find_closest_match(client, '', '', msg, nickname, number)
        return profile
    elif method == 1:
        classification = classify(msg, matched_agent, state)
        if classification[0] == "Chat":
            return "Chat"
        # 4/2 수정 -eunjeong 
        elif classification[0] == 'Connect':
            return 'Connect'
        elif classification[0] == 'Payment':
            return 'Payment'
        else:
            profile = find_closest_match(classification[0], classification[1], msg, nickname, number)
            return profile

In [15]:
def chat(msg, state):
    chat_prompt = get_chat_prompt(msg)
    state.append({"role": "system", "content": chat_prompt})
    response = client.chat.completions.create(
                    model="gpt-4-1106-preview",
                    messages=state,
                    stream=False)
    response = response.choices[0].message.content
    result_ME = { "ai_flag": "M", "type": 'chat', "mem_nick_name": '', "product_name": '', "product_price": 0, "talk_msg": msg}
    result_AI = { "ai_flag": "A", "type": 'chat', "mem_nick_name": '', "product_name": '', "product_price": 0, "talk_msg": response}
    state.append({"role": "assistant", "content": response})
    return result_ME, result_AI, state

In [16]:
def final_process(state, matched_agent, matched_msg, user_name):
    matched_agent = matched_agent[-14:]
    matched_msg = matched_msg[-14:]
    del state[0]
    state = state[-9:]
    state_message = {"role": "system",
                 "content": f"""You are a matchmaker, And the user's name is {user_name}.
                 Remember this, Refer to the chatting history below, and Answer the user's message in Korean."""}
    state.insert(0, state_message)
    return state, matched_agent, matched_msg

In [17]:
## 4/2 수정
def find_agent(msg, profile):
    RAG_prompt = matching_rag_prompt(msg, profile)
    response = client.chat.completions.create(
                    model = "gpt-4-1106-preview",
                    messages=[{"role": "system", "content": RAG_prompt}],
                    temperature = 0
                )
    response = response.choices[0].message.content
    # return response
    if "Agent: " in response and "Product: " not in response:
        agent = response[response.find("Agent")+7:]
        return agent
    elif "Agent: " in response and "Product: " in response:
        agent = response[response.find("Agent")+7:response.find("Product:")-2]
        return agent
    elif response == "X":
        return "X"
    elif len(response) > 50:
        return "X"
    else:
        return response[7:]

# m5 s1 ss2
# 추출된 Agent의 답변형식
def agent_print_form(agent, profile):
    for i in profile:
        if i[0] == agent:
            msg = i[1]
            text = f"""사용자의 요구사항을 바탕으로 다음 에이전트를 추천합니다.\n\nAgent: {agent}\n메시지: {msg}"""
            return text

# m5 s1
def Matching_print(profile, msg):
    print_agent = find_agent(msg, profile)
    if print_agent == 'X':
        final_answer = "죄송합니다, 현재 요청하신 매칭 조건에 적합한 사용자를 찾지 못하였습니다. 원하시는 다른 매칭이 있으신가요?"
    else:
        final_answer = agent_print_form(print_agent, profile)
    return final_answer

In [18]:
## 4/1 추가
def get_keyword(method, response, matched_agent, matched_msg):
    sub_matched_agent = re.findall(r'Agent: (.*?)\n', response)
    for i in range(len(sub_matched_agent)):
        matched_agent.append(sub_matched_agent[i])        
    if method == 0:
        sub_matched_msg = re.findall(r'메시지: (.*?)$', response)
        for i in range(len(sub_matched_msg)):
            matched_msg.append(sub_matched_msg[i])
    else:
        sub_matched_msg = re.findall(r'메시지: (.*?)\n', response)
        for i in range(len(sub_matched_msg)):
            matched_msg.append(sub_matched_msg[i])
    message = purchase_keyword(sub_matched_msg[0])
    return message, matched_agent, matched_msg

In [19]:
# input 값 추가
def printing(method, profile, msg, state, matched_agent, matched_msg):
    unmatch_msg = "죄송합니다, 현재 요청하신 매칭 조건에 적합한 사용자를 찾지 못하였습니다. 원하시는 다른 매칭이 있으신가요?"
    if method == 0:
        response =  Matching_print(profile, msg)
        if response == unmatch_msg:
            result_ME = { "ai_flag": "M", "type": 'chat', "mem_nick_name": '', "product_name": '', "product_price": 0, "talk_msg": msg}
            result_AI = { "ai_flag": "A", "type": 'chat', "mem_nick_name": '', "product_name": '', "product_price": 0, "talk_msg": response}
        else:
            message, matched_agent, matched_msg = get_keyword(method, response, matched_agent, matched_msg)
            if message == 'no_product':
                result_ME = { "ai_flag": "M", "type": 'match', "mem_nick_name": '', "product_name": '', "product_price": 0, "talk_msg": msg}
                result_AI = { "ai_flag": "A", "type": 'match', "mem_nick_name": '', "product_name": '', "product_price": 0, "talk_msg": response}
            else:
                result_ME = { "ai_flag": "M", "type": 'match', "mem_nick_name": '', "product_name": '', "product_price": 0, "talk_msg": msg}
                result_AI = { "ai_flag": "A", "type": 'match', "mem_nick_name": matched_agent[-1], "product_name": message[0], "product_price": message[1], "talk_msg": response}
        state.append({"role": "system", "content": f"매칭리스트: {profile}"})
        state.append({"role": "assistant", "content": response})
        return result_ME, result_AI, state, matched_agent, matched_msg

    elif method == 1:
        state.append({"role": "system", "content": RAG_prompt + f" 사용자: <{msg}>. 매칭리스트: {profile}"})
        response = client.chat.completions.create(
                        model="gpt-4-1106-preview",
                        messages=state,
                        stream=False)
        response = response.choices[0].message.content
        del state[-1]
        state.append({"role": "system", "content": f"매칭리스트: {profile}"})
        state.append({"role": "assistant", "content": response})
        if response == unmatch_msg:
            result_ME = { "ai_flag": "M", "type": 'chat', "mem_nick_name": '', "product_name": '', "product_price": 0, "talk_msg": msg}
            result_AI = { "ai_flag": "A", "type": 'chat', "mem_nick_name": '', "product_name": '', "product_price": 0, "talk_msg": response}
        else:
            message, matched_agent, matched_msg = get_keyword(method, response, matched_agent, matched_msg)
            if message == 'no_product':
                result_ME = { "ai_flag": "M", "type": 'match', "mem_nick_name": '', "product_name": '', "product_price": 0, "talk_msg": msg}
                result_AI = { "ai_flag": "A", "type": 'match', "mem_nick_name": '', "product_name": '', "product_price": 0, "talk_msg": response}
            else:
                result_ME = { "ai_flag": "M", "type": 'match', "mem_nick_name": '', "product_name": '', "product_price": 0, "talk_msg": msg}
                result_AI = { "ai_flag": "A", "type": 'match', "mem_nick_name": matched_agent[-1], "product_name": message[0], "product_price": message[1], "talk_msg": response}
        return result_ME, result_AI, state, matched_agent, matched_msg
    
    elif method == 2:
        one_agent = find_agent(msg, profile)
        if one_agent == "X":
            result_ME = { "ai_flag": "M", "type": 'chat', "mem_nick_name": '', "product_name": '', "product_price": 0, "talk_msg": msg}
            result_AI = { "ai_flag": "A", "type": 'chat', "mem_nick_name": '', "product_name": '', "product_price": 0, "talk_msg": unmatch_msg}
        else:
            for i in profile:
                if i[0] == one_agent:
                    agent_msg = i[1]
            state.append({"role": "system", 
                    "content": f"""
                        Quickly match an agent from a provided list with specific criteria. Here's what you need to do:
                        1. {one_agent}: Agent
                        2. {agent_msg}: 메시지
                        3. Answer the <매칭 이유> based on the information above.

                        Format your response as follows:
                        사용자의 요구사항을 바탕으로 다음 에이전트를 추천합니다.

                        Agent: [Recommended Agent]
                        메시지: [Agent's Message]
                        매칭 이유: [Reason for Match]

                        *Cautions:
                        - Only include one recommended agent.
                        - Must answer the reason for matching as concisely as possible in a single sentence. (100 characters less)"""})
            response = client.chat.completions.create(
                            model="gpt-4-1106-preview",
                            messages=state,
                            stream=False)
            response = response.choices[0].message.content

            del state[-1]
            state.append({"role": "system", "content": f"매칭리스트: {profile}"})
            state.append({"role": "assistant", "content": response})

            if response == unmatch_msg: 
                result_ME = { "ai_flag": "M", "type": 'chat', "mem_nick_name": '', "product_name": '', "product_price": 0, "talk_msg": msg}
                result_AI = { "ai_flag": "A", "type": 'chat', "mem_nick_name": '', "product_name": '', "product_price": 0, "talk_msg": response}
            else:
                message, matched_agent, matched_msg = get_keyword(method, response, matched_agent, matched_msg)
                if message == 'no_product':
                    result_ME = { "ai_flag": "M", "type": 'match', "mem_nick_name": '', "product_name": '', "product_price": 0, "talk_msg": msg}
                    result_AI = { "ai_flag": "A", "type": 'match', "mem_nick_name": '', "product_name": '', "product_price": 0, "talk_msg": response}
                else:    
                    result_ME = { "ai_flag": "M", "type": 'match', "mem_nick_name": '', "product_name": '', "product_price": 0, "talk_msg": msg}
                    result_AI = { "ai_flag": "A", "type": 'match', "mem_nick_name": matched_agent[-1], "product_name": message[0], "product_price": message[1], "talk_msg": response} 
        return result_ME, result_AI, state, matched_agent, matched_msg

In [20]:
## 4/1 오후 추가
def connect_sale(msg, state, profile, matched_agent, matched_msg):
    if profile == 'Connect':
        message = purchase_keyword(matched_msg[-1])
        if message == 'no_product':
            result_ME = { "ai_flag": "M", "type": 'connect', "mem_nick_name": matched_agent[-1], "product_name": '', "product_price": 0, "talk_msg": msg}
            result_AI = { "ai_flag": "A", "type": 'connect', "mem_nick_name": matched_agent[-1], "product_name": '', "product_price": 0, "talk_msg": f"네 {matched_agent[-1]} 연결해드리겠습니다."}
        else:
            result_ME = { "ai_flag": "M", "type": 'connect', "mem_nick_name": matched_agent[-1], "product_name": message[0], "product_price": message[1], "talk_msg": msg}
            result_AI = { "ai_flag": "A", "type": 'connect', "mem_nick_name": matched_agent[-1], "product_name": message[0], "product_price": message[1], "talk_msg": f"네 {matched_agent[-1]} 연결해드리겠습니다."}
    elif profile == 'Payment':
        message = purchase_keyword(matched_msg[-1])
        if message == 'no_product':
            result_ME = { "ai_flag": "M", "type": 'connect', "mem_nick_name": matched_agent[-1], "product_name": '', "product_price": 0, "talk_msg": msg}
            result_AI = { "ai_flag": "A", "type": 'connect', "mem_nick_name": matched_agent[-1], "product_name": '', "product_price": 0, "talk_msg": "결제할 상품이 없습니다."}
        else:
            result_ME = { "ai_flag": "M", "type": 'connect', "mem_nick_name": matched_agent[-1], "product_name": message[0], "product_price": message[1], "talk_msg": msg}
            result_AI = { "ai_flag": "A", "type": 'connect', "mem_nick_name": matched_agent[-1], "product_name": message[0], "product_price": message[1], "talk_msg": "결제를 진행합니다"}
    state.append({"role": "user", "content": msg})
    state.append({"role": "assistant", "content": result_AI['talk_msg']})
    return result_ME, result_AI, state

In [21]:
def process_user_message(msg, user_name, state, matched_agent, matched_msg):
    number = [1, 1, 1, 5]
    classify_method = 1
    print_method = 0

    while True:
        msg = input("사용자: ")
        if msg == "exit":
            return state
        route, result_ME, result_AI, state = message_process(msg, state, matched_agent, matched_msg)
        if route == "next":
            profile = make_profile(user_name, classify_method, msg, matched_agent, number, state)
            if profile == "Chat":
                result_ME, result_AI, state = chat(msg, state)
            elif profile == "Connect" or profile == "Payment":
                result_ME, result_AI, state = connect_sale(msg, state, profile, matched_agent, matched_msg)
            else:
                result_ME, result_AI, state, matched_agent, matched_msg = printing(print_method, profile, msg, state, matched_agent, matched_msg)
        state, matched_agent, matched_msg = final_process(state, matched_agent, matched_msg, user_name)
        print(result_ME, result_AI)
    return state, profile

In [22]:
user_name = "사용자"
state = [{"role": "system",
                 "content": f"""You are a matching agent, And the user's name is {user_name}.
                 Remember this, Refer to the chatting history below, and Answer the user's message in Korean."""}]

In [23]:
process_user_message('안녕하세요 ', user_name, state, ['짠짠'], ['집가고싶다'])

[{'role': 'system', 'content': "You are a matching agent, And the user's name is 사용자.\n                 Remember this, Refer to the chatting history below, and Answer the user's message in Korean."}, {'role': 'user', 'content': '진미채 사고 싶어 '}]
['Buy&Sell', '진미채']
{'ai_flag': 'M', 'type': 'match', 'mem_nick_name': '', 'product_name': '', 'product_price': 0, 'talk_msg': '진미채 사고 싶어 '} {'ai_flag': 'A', 'type': 'match', 'mem_nick_name': '반찬의달인', 'product_name': '진미채 500g', 'product_price': 15000, 'talk_msg': '사용자의 요구사항을 바탕으로 다음 에이전트를 추천합니다.\n\nAgent: 반찬의달인\n메시지: 반찬의달인님이 중부시장테스트가맹점에서 진미채 500g을 1개 당 15000원에 판매하고 있습니다.'}
[{'role': 'assistant', 'content': '사용자의 요구사항을 바탕으로 다음 에이전트를 추천합니다.\n\nAgent: 반찬의달인\n메시지: 반찬의달인님이 중부시장테스트가맹점에서 진미채 500g을 1개 당 15000원에 판매하고 있습니다.'}, {'role': 'user', 'content': '개별채팅'}]
['Connect', '개별채팅']
{'ai_flag': 'M', 'type': 'connect', 'mem_nick_name': '반찬의달인', 'product_name': '진미채 500g', 'product_price': 15000, 'talk_msg': '개별채팅'} {'ai_flag': 'A', 'type': 'connect', 'mem_n

KeyboardInterrupt: Interrupted by user

In [ ]:
profile = [('반찬의달인', '반찬의달인님이 중부시장테스트가맹점에서 진미채 500g을 1개 당 15000원에 판매하고 있습니다.'), ('대박쭈꾸미', '대박쭈꾸미님이 중부시장테스트가맹점13에서 젤리 100g을 1개 당 8000원에 판매하고 있습니다.'), ('똘이상회', '똘이상회님이 똘이상회에서 한입에 쏙 젤리(1kg)을 1개 당 10000원에 판매하고 있습니다.'), ('대유수산', '대유수산님이 대유수산에서 햇곱창돌김(100장) 450g을 1개 당 27900원에 판매하고 있습니다.')]

In [ ]:
find_agent('김 사고싶어', profile)

'대유수산'